### ChatGLM2-6B

前置依赖

In [ ]:
pip install protobuf transformers==4.30.2 cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate

运行模型

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("model/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("model/chatglm2-6b", trust_remote_code=True).half().cuda()
model = model.eval()

In [ ]:
for response, history in model.stream_chat(tokenizer, "你的任务是什么?", history=[]):
    print(response)

### CCGPTModel

In [ ]:
import sys
from CC.predictor.chatglm import GPTPredict

predictor = GPTPredict(model_name="ChatGLM2-6B", model_from_pretrained="model/chatglm3-6b")

In [ ]:
res = predictor("你好?", history=[])
print(res['data'])

In [ ]:
for res in predictor.predict_stream("你的任务是什么?", history=[]):
    sys.stdout.write('\r' + res['data'])
    sys.stdout.flush()

In [ ]:
with open('./a.txt', encoding='utf-8') as f:
    ask_content = f.read()
res = predictor(ask_content, history=[])
print(res['data'])

### ChatGLM_LoRA

In [ ]:
from CC.trainer.chatglm_lora import Trainer
from transformers import AutoTokenizer, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("model/chatglm2-6b", trust_remote_code=True)
config = AutoConfig.from_pretrained("model/chatglm2-6b", trust_remote_code=True)
trainer = Trainer(tokenizer=tokenizer, config=config, from_pretrained='./model/chatglm3-6b', loader_name='ChatGLM_LoRA', data_path='BossCondition', batch_size=2, task_name='BossCondition')

In [ ]:
for i in trainer(num_epochs=5):
    a = i

##### 预测文本

In [ ]:
from CC.predictor.chatglm_lora import Predictor

pred = Predictor(model_from_pretrained='./model/chatglm3-6b', resume_path='./save_model/BossCondition/ChatGLM_37038')

In [ ]:
result = pred('Instrcution: 请识别该商品的要素: 理光（Ricoh） M2700/M2701/2702多功能黑白激光复合机 a3复合机打印机一体机办公 M 2702(网络+双面+输稿器+7寸触屏) 官方标配\n Answer:', max_length=512)
print(result)

In [ ]:
with open('./a.txt', encoding='utf-8') as f:
    ask_content = f.read()
result = pred(ask_content, max_length=512)
print(result)

##### 预测商品要素

In [ ]:
from CC.predictor.chatglm_lora import Predictor

pred = Predictor(model_from_pretrained='./model/chatglm3-6b', resume_path='./save_model/BossCondition/ChatGLM_37038')

In [ ]:
result = pred('Instrcution: 请识别该商品的要素: 理光（Ricoh） M2700/M2701/2702多功能黑白激光复合机 a3复合机打印机一体机办公 M 2702(网络+双面+输稿器+7寸触屏) 官方标配\n Answer:', max_length=512)
print(result)

In [ ]:
import json
from tqdm import tqdm

with open('./data/Boss/BertPred/复印机.tsv', encoding='utf-8') as f:
    ori_list = f.read().split('\n')

if ori_list[-1] == '':
    ori_list = ori_list[:-1]

result = []
iter = tqdm(ori_list)
for item in iter:
    item = item.split('\t')
    res = pred(f'Instrcution: 请识别该商品的要素: {item[2]}\n Answer:', max_length=512)
    res_item = {
        'pred': res
    }
    answer_index = res.find('Answer:')
    iter.set_postfix(pred=json.dumps(res[answer_index + 7:], ensure_ascii=False))
    result.append(res_item)

with open('./data_record/BertPred_ChatGLMLoRA/复印机.json', 'w', encoding='utf-8') as f:
    for item in result:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

### 使用ChatGLM3-6B对商品进行要素抽取

In [ ]:
import sys
from CC.predictor.chatglm import GPTPredict

predictor = GPTPredict(model_name="ChatGLM2-6B", model_from_pretrained="model/chatglm3-6b")

In [ ]:
import json
from tqdm import tqdm

with open('./data/Boss/train.json') as f:
    ori_data = f.read().split('\n')

if ori_data[-1] == '':
    ori_data.pop()

result = []
for item in tqdm(ori_data):
    data = json.loads(item)
    item_id = data['item_id']
    context = data['context']
    question = context.replace('\n Answer: ', '')
    res = predictor.generate(question, max_length=1024)
    result.append({'item_id': item_id, 'question': question, 'answer': res})

with open('./data/Boss/train_result.json', encoding='utf-8', mode='w') as f:
    for item in result:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')